In [10]:
import os
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn import metrics
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.covariance import EmpiricalCovariance, EllipticEnvelope, MinCovDet
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

from scipy.cluster.hierarchy import dendrogram, fcluster, cophenet, set_link_color_palette
from scipy.spatial.distance import squareform, mahalanobis
from fastcluster import linkage, pdist

# saving models
from sklearn.externals import joblib

# incase we want to try some cleaning steps to see if it improves the model
import Clean_Function_Helpers as cfh

In [2]:
plt.rcParams['figure.figsize'] = (9,6)
sns.set_style('darkgrid')

SEED = 1111

## Overview

Taking two different approaches. 

    1. Try to model the difference between real and fraudulent charges.
        - Classifiers like Logistic Regression, NaiveBayes, Tree Ensembles etc
        - Sampling approaches over vs undersampling
    2. Try to identify core boundary of real charges and identify anything outside this boundary as fraudulent.
        - Covariance estimates, Local Outlier Factor, Clustering, One Class SVM, Hierarchical Clustering, 
        Model-based bayesian clustering.
        
This notebook focuses on the second approach: using robust statistical methods as well as un-supervised learning to identify outliers.

### Read Data

In [3]:
df = pd.read_csv('creditcard.csv')
df.Class.value_counts()/df.Class.value_counts().sum()

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [4]:
# We will test different transforms of the data

sub_cols = df.columns.drop(['Time', 'Class'])

scaled_df = cfh.scale_data(df, MinMaxScaler(), sub_cols)
deskewed = cfh.deskew_df(scaled_df, topn=10)

In [5]:
x = df[sub_cols]
y = df.Class


### Minimum Covariance Determinant

MCD is a method to compute a robust estimate for mean and covariance of a multivariate gaussian distributed dataset.
Empirically computing mean and covariance is known to be very sensitive to outliers, so MCD finds a _core subset_ of the data that best represents the underlying distribution. From these robust estimates, we can determine outliers by a points (usually Mahalanobis) distance to the robust mean.

_NOTE Sklearn spits continually spits out RuntimeWarnings on the regular dataset leading me to believe that the data is not well approximated by a normal distribution. Running the Elliptic Envelope on scaled-deskewed data seems to solve the issue._

In [79]:
x = deskewed[sub_cols]

params = {
    'contamination': [0.001,0.005, 0.01 0.1],
    'support_fraction': [0.6, 0.7, 0.8, 0.9]
}

def scorer(y, ypred):
    "Calculate F1 Score for Elliptic Envelope labels"
    ypred = np.where(ypred>0, 0, 1)
    return metrics.f1_score(y,ypred)
scorer = metrics.make_scorer(scorer)

# THIS TAKES 2 HOURS TO RUN
ee = EllipticEnvelope(assume_centered=True,random_state=SEED)
grid = GridSearchCV(ee, params, scorer, cv=4)
grid.fit(x,y)

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional i

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional i

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional i

GridSearchCV(cv=4, error_score='raise',
       estimator=EllipticEnvelope(assume_centered=True, contamination=0.1, random_state=1111,
         store_precision=True, support_fraction=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'contamination': [0.001, 0.005, 0.01], 'support_fraction': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(scorer), verbose=0)

In [80]:
print(grid.best_score_)
print(grid.best_params_)


0.42045984408933085
{'contamination': 0.001, 'support_fraction': 0.7}


In [81]:
grid.best_estimator_

EllipticEnvelope(assume_centered=True, contamination=0.001, random_state=1111,
         store_precision=True, support_fraction=0.7)

In [52]:
x = deskewed[sub_cols]

ee = EllipticEnvelope(assume_centered=True, contamination=0.001, support_fraction=0.7,random_state=SEED)
ee.fit(x,y)
ypred = np.where(ee.predict(x)>0, 0, 1)
print(metrics.f1_score(y,ypred))
print(metrics.recall_score(y,ypred))
print(metrics.precision_score(y,ypred))

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


0.5456885456885457
0.43089430894308944
0.743859649122807


We can use predictions as features, or maybe even better would be to feed in Mahalanobis distances from best mcd. Or we can create a Multivariate Normal Distribution and use it's pdf to generate features for rows. Although I believe this is the same as the Mahalanobis distance just scaled.

### Local Outlier Factor

Local Outlier Factor is a way of scoring data points based on their relative densities to their nearest neighbors.
The theory is that a “normal" data point is expected to have a similar density to it’s neighbors, while data points with lower relative density (as compared to their neighbors) are more likely to be outliers. 

See for example below, points O1, O2, and O3 are outliers, but point O4 is not even though it's _distance_ to it's neighbors is comparable to O1 and O2. However the density of O4s neighbors is _not_ comparable to the neighbors of O1 and O2.

![LOFExample](https://i.stack.imgur.com/EFB37.jpg![image.png](attachment:image.png)

In [6]:
def lof_grid(x, y, n_neighbors_opts):
    best_f1 = {'nn':0, 'thresh':0, 'f1':0, 'recall':0, 'precision':0}
    best_recall = {'nn':0, 'thresh':0, 'f1':0, 'recall':0, 'precision':0}
    best_precision = {'nn':0, 'thresh':0, 'f1':0, 'recall':0, 'precision':0}
    
    for nn in n_neighbors:
        lof = LocalOutlierFactor(nn)
        lof.fit(x,y)
        for thresh in np.arange(-1,-3, -0.2):
            ypred = np.where(lof.negative_outlier_factor_ < thresh, 1, 0)
            f1 = metrics.f1_score(y,ypred)
            recall = metrics.recall_score(y,ypred)
            precision = metrics.precision_score(y,ypred)
            if f1 > best_f1['f1']:
                best_f1 = {'nn':nn, 'thresh':thresh, 'f1':f1, 'recall':recall, 'precision':precision}
            if recall > best_recall['recall']:
                best_recall = {'nn':nn, 'thresh':thresh, 'f1':f1, 'recall':recall, 'precision':precision}
            if precision > best_precision['precision']:
                best_precision = {'nn':nn, 'thresh':thresh, 'f1':f1, 'recall':recall, 'precision':precision}
    return best_f1, best_recall, best_precision

Distance Based methods are very costly, so performing the rest on sub sample of data

In [7]:
# Interestingly, this algorithm performs way better on _unscaled_ data
# This is very strange and suggests that dollar amount is a far more important
# factor than the other variables
normal_samp = df[df.Class==0].sample(25000, random_state=SEED)
outliers = df[df.Class==1]
samp = pd.concat([normal_samp, outliers])
samp.Class.value_counts()

0    25000
1      492
Name: Class, dtype: int64

In [8]:
x = samp[sub_cols]
y = samp.Class

In [133]:
n_neighbors = np.arange(230,300, 5)
best_f1, best_recall, best_precision = lof_grid(x,y, n_neighbors)

print(best_f1)
print()
print(best_recall)
print()
print(best_precision)

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


{'nn': 255, 'thresh': -1.9999999999999998, 'f1': 0.5612153708668455, 'recall': 0.6382113821138211, 'precision': 0.5007974481658692}

{'nn': 285, 'thresh': -1.0, 'f1': 0.04986206191887198, 'recall': 0.991869918699187, 'precision': 0.025573839220207527}

{'nn': 295, 'thresh': -2.8, 'f1': 0.4625158831003813, 'recall': 0.3699186991869919, 'precision': 0.6169491525423729}


In [15]:
x = df[sub_cols]
y = df.Class

In [122]:
# Test on full dataset
lof = LocalOutlierFactor(best_f1['nn']) # use best f1 params
lof.fit(x)
for thresh in np.arange(-1,-3, -0.2):
    print('Thresh:', thresh)
    ypred = np.where(lof.negative_outlier_factor_ < thresh, 1, 0)
    f1 = metrics.f1_score(y,ypred)
    recall = metrics.recall_score(y,ypred)
    precision = metrics.precision_score(y,ypred)
    print(f1)
    print(recall)
    print(precision)
    print()

Thresh: -1.0
0.004118191235841053
0.9817073170731707
0.002063423574293929

Thresh: -1.2
0.014377209953966243
0.8760162601626016
0.007248082873671465

Thresh: -1.4
0.04573458400399622
0.8373983739837398
0.023509272467902995

Thresh: -1.5999999999999999
0.1151016875811337
0.8109756097560976
0.061946902654867256

Thresh: -1.7999999999999998
0.20505920344456405
0.774390243902439
0.11817617866004963

Thresh: -1.9999999999999998
0.269120654396728
0.6686991869918699
0.16845878136200718

Thresh: -2.1999999999999997
0.30254957507082153
0.5426829268292683
0.20974076983503534

Thresh: -2.3999999999999995
0.31601731601731603
0.4451219512195122
0.24496644295302014

Thresh: -2.5999999999999996
0.288695652173913
0.33739837398373984
0.25227963525835867

Thresh: -2.8
0.280561122244489
0.2845528455284553
0.2766798418972332



In [123]:
# Test on full dataset
lof = LocalOutlierFactor(best_precision['nn']) # use best precision params
lof.fit(x)
for thresh in np.arange(-1,-3, -0.2):
    print('Thresh:', thresh)
    ypred = np.where(lof.negative_outlier_factor_ < thresh, 1, 0)
    f1 = metrics.f1_score(y,ypred)
    recall = metrics.recall_score(y,ypred)
    precision = metrics.precision_score(y,ypred)
    print(f1)
    print(recall)
    print(precision)
    print()

/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Thresh: -1.0
0.004139104704810854
0.983739837398374
0.0020739153722549547

Thresh: -1.2
0.014288325109239232
0.8739837398373984
0.007203042028912676

Thresh: -1.4
0.045669729880256194
0.8333333333333334
0.023478211074843956

Thresh: -1.5999999999999999
0.11221309152734485
0.8048780487804879
0.060310691440755404

Thresh: -1.7999999999999998
0.20497725448220497
0.7784552845528455
0.11802773497688752

Thresh: -1.9999999999999998
0.29537223340040236
0.7459349593495935
0.18414450577019567

Thresh: -2.1999999999999997
0.36098069900886803
0.7032520325203252
0.24280701754385964

Thresh: -2.3999999999999995
0.4065146579804561
0.6341463414634146
0.29913710450623204

Thresh: -2.5999999999999996
0.39588281868566905
0.508130081300813
0.324254215304799

Thresh: -2.8
0.3458646616541353
0.37398373983739835
0.32167832167832167



In [124]:
# So best params are
nn = 295, 
thresh = -2.4, 

### DBSCAN

DBSCAN is a density based clustering method that labels points as outliers if they don't meet a certain threshold to belong to any nearby cluster. Points that are not within at least `eps` distance to `min_samples` other points are labeled outliers.

In [9]:
def db_grid(x, y, eps_opts, min_samp_opts):
    best_f1 = {'eps':0, 'ms':0, 'f1':0, 'recall':0, 'precision':0}
    best_recall = {'nn':0, 'thresh':0, 'f1':0, 'recall':0, 'precision':0}
    best_precision = {'nn':0, 'thresh':0, 'f1':0, 'recall':0, 'precision':0}
    
    for eps in eps_opts:
        for mso in min_samp_opts:
            db = DBSCAN(eps, mso)
            db.fit(x)
                
            ypred = np.where(db.labels_< 0, 1, 0)
            f1 = metrics.f1_score(y,ypred)
            recall = metrics.recall_score(y,ypred)
            precision = metrics.precision_score(y,ypred)
            if f1 > best_f1['f1']:
                best_f1 = {'eps':eps, 'ms':mso, 'f1':f1, 'recall':recall, 'precision':precision}
            if recall > best_recall['recall']:
                best_recall = {'eps':eps, 'ms':mso, 'f1':f1, 'recall':recall, 'precision':precision}
            if precision > best_precision['precision']:
                best_precision = {'eps':eps, 'ms':mso, 'f1':f1, 'recall':recall, 'precision':precision}
    return best_f1, best_recall, best_precision

In [143]:
normal_samp = df[df.Class==0].sample(25000, random_state=SEED)
outliers = df[df.Class==1]
samp = pd.concat([normal_samp, outliers])
samp.Class.value_counts()

x = samp[sub_cols]
y = samp.Class

In [145]:
eps_opts = np.arange(10,25, 3)
ms_opts = [8,10,12,15]

best_f1, best_recall, best_precision = db_grid(x,y, eps_opts, ms_opts)
print(best_f1)
print()
print(best_recall)
print()
print(best_precision)

{'eps': 10, 'ms': 12, 'f1': 0.3737991266375546, 'recall': 0.4349593495934959, 'precision': 0.327718223583461}

{'eps': 10, 'ms': 15, 'f1': 0.36348408710217756, 'recall': 0.4410569105691057, 'precision': 0.3091168091168091}

{'eps': 10, 'ms': 8, 'f1': 0.34509803921568627, 'recall': 0.35772357723577236, 'precision': 0.3333333333333333}


TAKES TOO LONG --- WHAT DO I DO

In [ ]:
# # Test on full dataset
# x = df[sub_cols]
# y = df.Class


# for d in [best_f1, best_recall, best_precision]:
#     print(d)
#     eps = d['eps']
#     ms  = d['ms']
#     db = DBSCAN(eps, ms)
#     db.fit(x)
    
#     ypred = np.where(db.labels_ < 0, 1, 0)
#     f1 = metrics.f1_score(y,ypred)
#     recall = metrics.recall_score(y,ypred)
#     precision = metrics.precision_score(y,ypred)
#     print(f1)
#     print(recall)
#     print(precision)
#     print()

{'eps': 10, 'ms': 12, 'f1': 0.3737991266375546, 'recall': 0.4349593495934959, 'precision': 0.327718223583461}


### Isolation Forests

In [16]:
x = df[sub_cols]
y = df.Class

In [27]:
for n in [200, 250, 300, 350]:
    print(n)
    isf = IsolationForest(n, max_samples=0.333, max_features=0.6, contamination=0.005)
    isf.fit(x)
    pred = isf.predict(x)
    ypred = np.where(pred< 0, 1, 0)
    f1 = metrics.f1_score(y,ypred)
    recall = metrics.recall_score(y,ypred)
    precision = metrics.precision_score(y,ypred)
    print(f1)
    print(recall)
    print(precision)
    print()

200


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


0.3077725612936881
0.5995934959349594
0.20701754385964913

250


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


0.2806468440271257
0.5467479674796748
0.1887719298245614

300


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


0.2785602503912363
0.5426829268292683
0.18736842105263157

350


/Users/rick.shapiro/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


0.28482003129890454
0.5548780487804879
0.19157894736842104



In [24]:
grid.best_score_, grid.best_params_

(0.21858724218519207, {'contamination': 0.005, 'n_estimators': 250})

In [20]:
ypred = np.where(isf.predict(x) <0, 1, 0)
f1 = metrics.f1_score(y,ypred)
recall = metrics.recall_score(y,ypred)
precision = metrics.precision_score(y,ypred)
print(f1)
print(recall)
print(precision)
print()

0.1981981981981982
0.1565040650406504
0.27017543859649124

